In [ ]:
%env CUDA_LAUNCH_BLOCKING=1
%env CUBLAS_WORKSPACE_CONFIG=:4096:8
%load_ext autoreload
%autoreload 2
from GraPL import hyperparameter_profiles, segment_bsds
# automatically reload modules when they have changed
import glob
import tqdm
import matplotlib.pyplot as plt
from GraPL.evaluate import bsds_score_directory
import numpy as np
import json
import os
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def run_experiment(base_params, changes, num_trials=2):
    results_base_dir = f'experiment_results/coldstart'
    aggregate_scores = {}
    for change in changes:
        change_name = change["use_cold_start"]
        print(f'Running experiment with use_cold_start={change_name}...')
        results_dir = f'{results_base_dir}/{change_name}'
        trials = []
        mean_over_trials = {}
        with tqdm.tqdm(total=num_trials * 200) as progress_bar:
            for trial_num in range(num_trials):
                params = base_params.copy()
                for key, value in change.items():
                    params[key] = value
                params['seed'] = trial_num
                trial_results_dir = f'{results_dir}/{trial_num}'
                trial_scores = segment_bsds(results_dir=trial_results_dir, progress_bar=progress_bar, **params)
                image_ids = list(trial_scores.keys())
                metrics = trial_scores[image_ids[0]].keys()
                trial_scores = {metric: np.mean([trial_scores[id][metric] for id in image_ids]) for metric in metrics}
                trials.append(trial_scores)
        for metric in trials[0]:
            mean_over_trials[metric] = np.mean([trial[metric] for trial in trials])
        aggregate_scores[change_name] = mean_over_trials
        print(f'Scores for use_cold_start={change_name}: {mean_over_trials["accuracy"]}')
    return aggregate_scores

base_params = hyperparameter_profiles["best_miou"]

changes = [{"use_cold_start": False}, {"use_cold_start": True}]

aggregate_scores = run_experiment(base_params, changes, num_trials=10)